In [ ]:
# pip install transformers torch
# pip install tf-keras
# pip install --upgrade tensorflow
# pip install --upgrade accelerate
# pip install transformers[torch]
# import accelerate
# print(accelerate.__version__)

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
result2 = pd.read_csv('community_add_market2.csv')

In [3]:
result2 = result2.drop_duplicates(subset=['내용_토큰'])

In [4]:
result2['label'].value_counts()

label
-2    37840
 0    16149
 1    12916
Name: count, dtype: int64

In [5]:
# 데이터 로드 및 전처리
# result2는 이미 로드된 DataFrame이라고 가정합니다.

sample_data_positive = result2[result2['label'] == 1].sample(12916)
sample_data_negative = result2[result2['label'] == 0].sample(12916)

total_data = pd.concat([sample_data_positive, sample_data_negative])
total_data = total_data.drop_duplicates(subset=['내용'])

# 학습 데이터와 테스트 데이터를 구분
train_data, test_data = train_test_split(total_data, test_size=0.25, random_state=2025)

# koBERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')

# 데이터 인코딩
def encode_data(data):
    return tokenizer(
        data['내용'].tolist(),
        padding=True,
        truncation=True,
        max_length=128,  # 필요에 따라 조정 가능
        return_tensors='pt'
    )

train_encodings = encode_data(train_data)
test_encodings = encode_data(test_data)

# 데이터셋 클래스 정의
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_data['label'].values)
test_dataset = SentimentDataset(test_encodings, test_data['label'].values)

# 모델 로드
model = BertForSequenceClassification.from_pretrained('kykim/bert-kor-base', num_labels=2)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,  # 배치 사이즈를 줄여 메모리 사용 최적화
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_strategy="epoch",
    save_strategy="epoch"
)

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 모델 훈련
trainer.train()

# 테스트 데이터 평가
test_results = trainer.evaluate()
print('\n 테스트 손실 : %.4f' % test_results['eval_loss'])
print('\n 테스트 정확도: %.4f' % test_results['eval_accuracy'])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at kykim/bert-kor-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 